# OpenAI Function calling

Source: https://platform.openai.com/docs/guides/gpt/function-calling

In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import openai
import json
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())  # add .env to .gitignore
openai.api_key = os.getenv("OPENAI_API_KEY")


In [ ]:
# Step 0: Define function
# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API

def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [ ]:
get_current_weather("boston", unit="fahrenheit")

'{"location": "boston", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [ ]:
# functions: A list of functions the model may generate JSON inputs for.
# name: string; Required; The name of the function to be called. Must be a-z, A-Z, 0-9, or contain underscores and dashes, with a maximum length of 64.
# description; string; Optional; The description of what the function does.
# parameters; object; Optional; The parameters the functions accepts, described as a JSON Schema object. See the guide for examples, and the JSON Schema reference for documentation about the format.
functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]

In [ ]:
# Step 1, send model the user query and what functions it has access to
# functions: A list of functions the model may generate JSON inputs for.
# function_call: Controls how the model responds to function calls.
    # "none" means the model does not call a function, and responds to the end-user.
    # "auto" means the model can pick between an end-user or calling a function.
    # Specifying a particular function via {"name":\ "my_function"} forces the model to call that function.
    # "none" is the default when no functions are present. "auto" is the default if functions are present.
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[{"role": "user", "content": "What's the weather like in Boston?"}],
    functions=functions,
    function_call="auto",
)

message = response["choices"][0]["message"]

In [ ]:
message

<OpenAIObject at 0x7fa9d14422a0> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}

In [ ]:
# Step 2, check if the model wants to call a function
if message.get("function_call"):
    function_name = message["function_call"]["name"]

In [ ]:
function_name

'get_current_weather'

In [ ]:
# Step 3, call the function
# Note: the JSON response from the model may not be valid JSON
function_response = get_current_weather(
    location = eval(message['function_call']['arguments']).get("location")
)

In [ ]:
function_response

'{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'

In [ ]:
# Step 4, send model the info on the function call and function response
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": "What is the weather like in boston?"},
        message,
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        },
    ],
)

In [ ]:
second_response

<OpenAIObject chat.completion id=chatcmpl-7RBP5dSERFSegmFKmpf6OM14i0bU9 at 0x7fa9c0a97e20> JSON: {
  "id": "chatcmpl-7RBP5dSERFSegmFKmpf6OM14i0bU9",
  "object": "chat.completion",
  "created": 1686713047,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston, MA is sunny and windy with a temperature of 72\u00b0F."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 75,
    "completion_tokens": 19,
    "total_tokens": 94
  }
}

In [ ]:
second_response['choices'][0]['message']['content']

'The current weather in Boston, MA is sunny and windy with a temperature of 72°F.'

# Convert LangChain tools to OpenAI functions

Source: https://python.langchain.com/en/latest/modules/agents/tools/tools_as_openai_functions.html

In [ ]:
!pip install langchain

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage
model = ChatOpenAI(model="gpt-3.5-turbo-0613")
from langchain.tools import MoveFileTool, format_tool_to_openai_function


In [ ]:
??MoveFileTool

In [ ]:
tools = [MoveFileTool()]
tools

[MoveFileTool(name='move_file', description='Move or rename a file from one location to another', args_schema=<class 'langchain.tools.file_management.move.FileMoveInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, root_dir=None)]

In [ ]:
functions = [format_tool_to_openai_function(t) for t in tools]
functions

[{'name': 'move_file',
  'description': 'Move or rename a file from one location to another',
  'parameters': {'title': 'FileMoveInput',
   'description': 'Input for MoveFileTool.',
   'type': 'object',
   'properties': {'source_path': {'title': 'Source Path',
     'description': 'Path of the file to move',
     'type': 'string'},
    'destination_path': {'title': 'Destination Path',
     'description': 'New path for the moved file',
     'type': 'string'}},
   'required': ['source_path', 'destination_path']}}]

In [ ]:
message = model.predict_messages(
    [HumanMessage(content='move file foo to bar')],
    functions=functions)

In [ ]:
message

AIMessage(content='', additional_kwargs={'function_call': {'name': 'move_file', 'arguments': '{\n  "source_path": "foo",\n  "destination_path": "bar"\n}'}}, example=False)

In [ ]:
message.additional_kwargs['function_call']

{'name': 'move_file',
 'arguments': '{\n  "source_path": "foo",\n  "destination_path": "bar"\n}'}